<font color='green'>***Installation and Libraries Import***</font>
---
--- 

In [1]:
%pip install flwr==1.5.0
%pip install 'flwr[simulation]==1.5.0'
%pip install ray==2.31.0
%pip install --upgrade pip
%pip install torch torchvision matplotlib
%pip install async-timeout

Note: you may need to restart the kernel to use updated packages.
  Using cached pydantic-1.10.24-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (154 kB)
INFO: pip is looking at multiple versions of flwr[simulation] to determine which version is compatible with other requirements. This could take a while.
  Using cached flwr-1.5.0-py3-none-any.whl.metadata (13 kB)
ERROR: Could not find a version that satisfies the requirement ray==2.6.3; extra == "simulation" (from flwr[simulation]) (from versions: 2.31.0, 2.32.0rc0, 2.32.0, 2.33.0, 2.34.0, 2.35.0, 2.36.0, 2.36.1, 2.37.0, 2.38.0, 2.39.0, 2.40.0, 2.41.0, 2.42.0, 2.42.1, 2.43.0, 2.44.0, 2.44.1, 2.45.0, 2.46.0, 2.47.0, 2.47.1, 2.48.0, 2.49.0, 2.49.1, 2.49.2, 2.50.0, 2.50.1, 2.51.0, 2.51.1, 2.51.2, 2.52.0, 2.52.1)
ERROR: No matching distribution found for ray==2.6.3; extra == "simulation"
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
libraries_to_uninstall = [
    "tb-nightly==2.18.0a20240701",
    "tensorboard==2.16.2",
    "tensorboard-data-server==0.7.2",
    "tensorboard-plugin-wit==1.8.1",
    "tensorflow==2.16.2",
    "tensorflow-io-gcs-filesystem==0.37.0",
    "termcolor==2.4.0",
    "terminado==0.18.1",
    "tf-estimator-nightly==2.8.0.dev2021122109",
    "tf_keras-nightly==2.18.0.dev2024070109",
    "tf-nightly==2.18.0.dev20240626"
]
for library in libraries_to_uninstall:
    os.system(f"pip uninstall -y {library}")

In [3]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
import torch.optim as optim
from torch.utils.data import Dataset, WeightedRandomSampler
import flwr as fl
from flwr.common import Metrics
# from flwr_datasets import FederatedDataset
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
import random
import math
import pickle
import csv
import copy
print(fl.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

1.5.0
cuda:0


<font color='Brown'>***FL Constants***</font>
---
--- 

In [4]:
# DEVICE
NUM_CLIENTS = 48
ROUNDS = 40
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
EPOCHS = 1
DATA_GROUPS = 40 
BATCH_ROUND = 6
SIZE_ROUND = int(BATCH_ROUND * BATCH_SIZE * NUM_CLIENTS)
PATH = 'GlobalOutputs'

<font color='Light Blue'>***Dataset Preparations***</font>
---
--- 

In [5]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Ensures reproducibility for CUDA and CPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    os.environ["PYTHONHASHSEED"] = str(seed)

In [6]:
def seed_worker(worker_id):
    worker_seed = 42
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [7]:
g = torch.Generator()
g.manual_seed(42)

In [8]:
set_seed(42)

In [9]:
TrafficData = {}
TrafficData['Dataset']={}
sets_names = ['30','100','70','50','120']
for  DATA_NUM in sets_names:
    TrafficData['Dataset'][DATA_NUM]=pd.read_csv(f'Datasets/2_Dataset_8_Attack_{DATA_NUM}_normal.csv', low_memory=False, quoting=csv.QUOTE_NONE, on_bad_lines='skip')
    print(DATA_NUM, TrafficData['Dataset'][DATA_NUM].shape)
for DATA_NUM in TrafficData['Dataset']:
    TrafficData['Dataset'][DATA_NUM]=TrafficData['Dataset'][DATA_NUM].sample(frac=1, random_state=42).reset_index(drop=True)

30 (184320, 99)
100 (184320, 99)
70 (184320, 99)
50 (184320, 99)
120 (120000, 99)


In [10]:
TrafficData['Split'] = {}
sets_training =  ['30','100','70','50']
for DATA_NUM in sets_training:
    TrafficData['Split'][DATA_NUM] = np.array_split(TrafficData['Dataset'][DATA_NUM],DATA_GROUPS)

TrafficData['Combined'] = pd.concat([TrafficData['Split']['30'][0], TrafficData['Split']['100'][0], TrafficData['Split']['70'][0], TrafficData['Split']['50'][0]]).reset_index(drop=True)
for GROUP in range(1, DATA_GROUPS):
    TrafficData['Combined'] = pd.concat([TrafficData['Combined'], TrafficData['Split']['30'][GROUP], TrafficData['Split']['100'][GROUP], TrafficData['Split']['70'][GROUP], TrafficData['Split']['50'][GROUP]]).reset_index(drop=True)
print(TrafficData['Combined'].shape)            

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


(737280, 99)


In [11]:
TrafficData['Train'] = {}
TrafficData['Train']['X'] = TrafficData['Combined'].iloc[:, 0:-1]
TrafficData['Train']['y'] = TrafficData['Combined'].iloc[:, -1]
print(TrafficData['Train']['X'].shape)
print(TrafficData['Train']['y'].shape)

TrafficData['Test'] = {}
TrafficData['Test']['X']=TrafficData['Dataset']['120'].iloc[:, 0:-1]
TrafficData['Test']['y']=TrafficData['Dataset']['120'].iloc[:, -1]
print(TrafficData['Test']['X'].shape)
print(TrafficData['Test']['y'].shape)

(737280, 98)
(737280,)
(120000, 98)
(120000,)


In [12]:
scaler = StandardScaler()
model = scaler.fit(TrafficData['Train']['X'])
TrafficData['Train']['X'] = model.transform(TrafficData['Train']['X'])
TrafficData['Test']['X'] = model.transform(TrafficData['Test']['X'])

TrafficData['Train']['X'], TrafficData['Train']['y']= np.array(TrafficData['Train']['X']), np.array(TrafficData['Train']['y'])
print(type(TrafficData['Train']['X']),type(TrafficData['Train']['y']))
print(TrafficData['Train']['X'].shape,TrafficData['Train']['y'].shape)
TrafficData['Test']['X'], TrafficData['Test']['y']= np.array(TrafficData['Test']['X']), np.array(TrafficData['Test']['y'])
print(type(TrafficData['Test']['X']),type(TrafficData['Test']['y']))
print(TrafficData['Test']['X'].shape,TrafficData['Test']['y'].shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(737280, 98) (737280,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(120000, 98) (120000,)


In [13]:
TrafficData['ROUNDS']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['ROUNDS'][ROUND]={}

SIZE_Demo = SIZE_ROUND
for ROUND in range(1,ROUNDS+1):
    if ROUND == 1:
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][:SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][:SIZE_Demo]
    else:
        print((SIZE_Demo - SIZE_ROUND),SIZE_Demo)
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
    SIZE_Demo = SIZE_Demo + SIZE_ROUND
for ROUND in TrafficData['ROUNDS']:
    print("ROUND: ", ROUND, TrafficData['ROUNDS'][ROUND]['X'].shape, TrafficData['ROUNDS'][ROUND]['y'].shape)
print(SIZE_Demo, SIZE_ROUND)

18432 36864
36864 55296
55296 73728
73728 92160
92160 110592
110592 129024
129024 147456
147456 165888
165888 184320
184320 202752
202752 221184
221184 239616
239616 258048
258048 276480
276480 294912
294912 313344
313344 331776
331776 350208
350208 368640
368640 387072
387072 405504
405504 423936
423936 442368
442368 460800
460800 479232
479232 497664
497664 516096
516096 534528
534528 552960
552960 571392
571392 589824
589824 608256
608256 626688
626688 645120
645120 663552
663552 681984
681984 700416
700416 718848
718848 737280
ROUND:  1 (18432, 98) (18432,)
ROUND:  2 (18432, 98) (18432,)
ROUND:  3 (18432, 98) (18432,)
ROUND:  4 (18432, 98) (18432,)
ROUND:  5 (18432, 98) (18432,)
ROUND:  6 (18432, 98) (18432,)
ROUND:  7 (18432, 98) (18432,)
ROUND:  8 (18432, 98) (18432,)
ROUND:  9 (18432, 98) (18432,)
ROUND:  10 (18432, 98) (18432,)
ROUND:  11 (18432, 98) (18432,)
ROUND:  12 (18432, 98) (18432,)
ROUND:  13 (18432, 98) (18432,)
ROUND:  14 (18432, 98) (18432,)
ROUND:  15 (18432, 98) (

In [14]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [15]:
TrafficData['trainsets']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['trainsets'][ROUND]= TabularDataset(torch.from_numpy(TrafficData['ROUNDS'][ROUND]['X']).float(), torch.from_numpy(TrafficData['ROUNDS'][ROUND]['y']).long())
TrafficData['testset'] = TabularDataset(torch.from_numpy(TrafficData['Test']['X']).float(), torch.from_numpy(TrafficData['Test']['y']).long())

In [16]:
def load_train(numberofclients, ROUND):    
    portion_size = int(BATCH_ROUND*BATCH_SIZE)
    num_portions = int(NUM_CLIENTS)
    portion_indices = []
    for i in range(num_portions):
        start_idx = i * portion_size
        end_idx = (i + 1) * portion_size
        portion_indices.append(list(range(start_idx, min(end_idx, SIZE_ROUND))))
    portion_datasets = [Subset(TrafficData['trainsets'][ROUND], indices) for indices in portion_indices]
    portion_loaders = [ DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False,worker_init_fn=seed_worker, generator=g) for dataset in portion_datasets]             
    return portion_loaders
def load_test(numberofclients):    
    testloader = DataLoader(TrafficData['testset'],batch_size=BATCH_SIZE,shuffle=False,worker_init_fn=seed_worker,generator=g)
    return testloader

In [17]:
Dataloaders = {}
for ROUND in range(1, ROUNDS+1):
    Dataloaders[ROUND] = load_train(NUM_CLIENTS, ROUND)
Dataloaders['Test'] = load_test(NUM_CLIENTS)

In [18]:
for i, batch in enumerate(Dataloaders['Test']):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64
Batch 7 size: 64
Batch 8 size: 64
Batch 9 size: 64
Batch 10 size: 64
Batch 11 size: 64
Batch 12 size: 64
Batch 13 size: 64
Batch 14 size: 64
Batch 15 size: 64
Batch 16 size: 64
Batch 17 size: 64
Batch 18 size: 64
Batch 19 size: 64
Batch 20 size: 64
Batch 21 size: 64
Batch 22 size: 64
Batch 23 size: 64
Batch 24 size: 64
Batch 25 size: 64
Batch 26 size: 64
Batch 27 size: 64
Batch 28 size: 64
Batch 29 size: 64
Batch 30 size: 64
Batch 31 size: 64
Batch 32 size: 64
Batch 33 size: 64
Batch 34 size: 64
Batch 35 size: 64
Batch 36 size: 64
Batch 37 size: 64
Batch 38 size: 64
Batch 39 size: 64
Batch 40 size: 64
Batch 41 size: 64
Batch 42 size: 64
Batch 43 size: 64
Batch 44 size: 64
Batch 45 size: 64
Batch 46 size: 64
Batch 47 size: 64
Batch 48 size: 64
Batch 49 size: 64
Batch 50 size: 64
Batch 51 size: 64
Batch 52 size: 64
Batch 53 size: 64
Batch 54 size: 64
Batch 55 size: 64
Batch 56 size: 64
B

In [19]:
for i, batch in enumerate(Dataloaders['Test']):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64
Batch 7 size: 64
Batch 8 size: 64
Batch 9 size: 64
Batch 10 size: 64
Batch 11 size: 64
Batch 12 size: 64
Batch 13 size: 64
Batch 14 size: 64
Batch 15 size: 64
Batch 16 size: 64
Batch 17 size: 64
Batch 18 size: 64
Batch 19 size: 64
Batch 20 size: 64
Batch 21 size: 64
Batch 22 size: 64
Batch 23 size: 64
Batch 24 size: 64
Batch 25 size: 64
Batch 26 size: 64
Batch 27 size: 64
Batch 28 size: 64
Batch 29 size: 64
Batch 30 size: 64
Batch 31 size: 64
Batch 32 size: 64
Batch 33 size: 64
Batch 34 size: 64
Batch 35 size: 64
Batch 36 size: 64
Batch 37 size: 64
Batch 38 size: 64
Batch 39 size: 64
Batch 40 size: 64
Batch 41 size: 64
Batch 42 size: 64
Batch 43 size: 64
Batch 44 size: 64
Batch 45 size: 64
Batch 46 size: 64
Batch 47 size: 64
Batch 48 size: 64
Batch 49 size: 64
Batch 50 size: 64
Batch 51 size: 64
Batch 52 size: 64
Batch 53 size: 64
Batch 54 size: 64
Batch 55 size: 64
Batch 56 size: 64
B

In [20]:
from collections import Counter
for CLUSTER in range (1, 9):
    DEVICE_PERCENTAGE = []
    for DEVICE__ in range(0, 12):
        for i, batch in enumerate(Dataloaders[CLUSTER][DEVICE__]):
            _, labels = batch
            class_counts = Counter(labels.numpy())
            total_records = sum(class_counts.values())
            class_0_count = class_counts.get(0, 0)
            percentage_class_0 = (class_0_count / total_records) * 100
            DEVICE_PERCENTAGE.append(percentage_class_0)
            # print(f"Batch {i+1}: {dict(class_counts)}")
            # print(f"Percentage of class 0: {percentage_class_0:.2f}%\n")
    # print(DEVICE_PERCENTAGE)        
    chunk_size = 6
    averages = [sum(DEVICE_PERCENTAGE[i:i + chunk_size]) / chunk_size for i in range(0, len(DEVICE_PERCENTAGE), chunk_size)]
    # print("Averages of every device:")
    # print(averages)
    chunk_size_4 = 4
    averages = [sum(averages[i:i + chunk_size_4]) / chunk_size_4 for i in range(0, len(averages), chunk_size_4)]
    # print("Averages of every 4 devices:")
    print(averages)

[30.338541666666664, 29.6875, 29.557291666666664]
[31.184895833333332, 27.473958333333332, 29.296875]
[31.640625, 29.4921875, 30.6640625]
[32.2265625, 28.515625, 29.361979166666668]
[28.776041666666668, 31.901041666666668, 31.119791666666668]
[29.947916666666664, 30.729166666666664, 29.427083333333336]
[30.46875, 31.25, 29.8828125]
[28.971354166666668, 29.557291666666668, 28.515625]


In [21]:
#del TrafficData

<font color='Red'>***Neural Network***</font>
---
--- 

In [22]:
class TabTransformer(nn.Module):
    def __init__(self, num_features, emb_dim=32, num_layers=3, num_heads=4, mlp_dim=64, num_classes=9):
        super().__init__()
        self.num_features = num_features
        self.emb_dim = emb_dim
        self.feature_projection = nn.Linear(1, emb_dim)
        self.feature_id_emb = nn.Parameter(torch.randn(num_features, emb_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=num_heads, dim_feedforward=mlp_dim, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_head = nn.Sequential(
            nn.Linear(emb_dim, mlp_dim),
            nn.ReLU(),
            nn.Linear(mlp_dim, num_classes)
        )
    def forward(self, x):
        x = x.unsqueeze(-1)
        emb = self.feature_projection(x)
        emb = emb + self.feature_id_emb.unsqueeze(0)
        z = self.transformer(emb)
        pooled = z.mean(dim=1)
        logits = self.output_head(pooled)
        return logits

In [23]:
num_features = TrafficData['Train']['X'].shape[1]
Random  = TabTransformer(num_features)
for param_tensor in Random.state_dict():
    print(param_tensor, "\t", Random.state_dict()[param_tensor].size())
torch.save(Random.state_dict(), "0_Input_Random_model_Tab.pth")

feature_id_emb 	 torch.Size([98, 32])
feature_projection.weight 	 torch.Size([32, 1])
feature_projection.bias 	 torch.Size([32])
transformer.layers.0.self_attn.in_proj_weight 	 torch.Size([96, 32])
transformer.layers.0.self_attn.in_proj_bias 	 torch.Size([96])
transformer.layers.0.self_attn.out_proj.weight 	 torch.Size([32, 32])
transformer.layers.0.self_attn.out_proj.bias 	 torch.Size([32])
transformer.layers.0.linear1.weight 	 torch.Size([64, 32])
transformer.layers.0.linear1.bias 	 torch.Size([64])
transformer.layers.0.linear2.weight 	 torch.Size([32, 64])
transformer.layers.0.linear2.bias 	 torch.Size([32])
transformer.layers.0.norm1.weight 	 torch.Size([32])
transformer.layers.0.norm1.bias 	 torch.Size([32])
transformer.layers.0.norm2.weight 	 torch.Size([32])
transformer.layers.0.norm2.bias 	 torch.Size([32])
transformer.layers.1.self_attn.in_proj_weight 	 torch.Size([96, 32])
transformer.layers.1.self_attn.in_proj_bias 	 torch.Size([96])
transformer.layers.1.self_attn.out_proj.w

In [24]:
def train(net, trainloader, epochs: int, verbose=True):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    for epoch in range(epochs):
        net.train()
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
            predictions = torch.max(outputs.data, 1)[1]
            prediction_matrix.append(predictions.tolist())
            actual_matrix.append(labels.tolist())
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        loss_matrix.append(epoch_loss.tolist())
        acc_matrix.append(epoch_acc)
    return prediction_matrix, actual_matrix, acc_matrix, loss_matrix
def test(net, testloader):
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(testloader.dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy)    
    print(f"Evaluation: eval loss {loss}, eval accuracy {accuracy}")
    return loss, accuracy, prediction_matrix, actual_matrix, acc_matrix, loss_matrix

In [25]:
prediction_dict= {}
actual_dict= {}
accuracy_dict= {}
loss_dict= {}

<font color='Brown'>***Federated Learning Classes***</font>
---
--- 

In [26]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [27]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, FL_Update):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.FL_Update = FL_Update
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        local_epochs = config["local_epochs"]
        print(f"[Client {self.cid}, round {self.FL_Update}] fit, config: {config}")
        set_parameters(self.net, parameters)
        _1, _2, _3, _4 = train(self.net, self.trainloader, epochs=local_epochs)
        prediction_dict[f'C{self.cid}R{self.FL_Update}'] = _1
        actual_dict[f'C{self.cid}R{self.FL_Update}'] = _2
        accuracy_dict[f'C{self.cid}R{self.FL_Update}'] = _3
        loss_dict[f'C{self.cid}R{self.FL_Update}'] = _4
        # Save model updates (parameters)
        # update_filename = f'EdgeCooperation/Performance/Results/C{self.cid}R{self.FL_Update}_update.pkl'
        # with open(update_filename, 'wb') as update_outfile:
        #     pickle.dump(get_parameters(self.net), update_outfile)        

        return get_parameters(self.net), len(self.trainloader), {}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}")
        print(f"[Client {self.cid}] evaluate, config: {config}")
        return None

In [28]:
def retrieve_and_sort_client_updates(global_model, round_number, client_ids):
    client_updates = {}
    for cid in client_ids:
        update_filename = f'EdgeCooperation/Performance/Results/C{cid}R{round_number}_update.pkl'
        with open(update_filename, 'rb') as update_file:
            client_update = pickle.load(update_file)
            client_updates[cid] = client_update
    client_contributions = {cid: calculate_weight_magnitude(global_model, update) for cid, update in client_updates.items()}
    sorted_clients = sorted(client_contributions.items(), key=lambda x: x[1], reverse=True)
    least_contributing_clients = sorted_clients[-3:]
    return sorted_clients, least_contributing_clients

def calculate_weight_magnitude(global_model, client_update):
    """
    Calculate the L2 norm of the weight difference between the global model and client's updated model.
    
    Args:
    global_model (nn.Module): The global model before client update.
    client_update (list): List of numpy arrays representing client's updated model parameters.

    Returns:
    float: The L2 norm of the weight difference.
    """
    weight_diff = 0.0
    global_parameters = [param.detach().cpu().numpy() for param in global_model.parameters()]
    for global_param, client_param in zip(global_parameters, client_update):
        weight_diff += np.linalg.norm(global_param - client_param)
    return weight_diff


<font color='Brown'>***Clients Functions***</font>
---
--- 

In [29]:
def General_Client():
    def client_fn(cid: int, Round: int) -> FlowerClient:
        clients_ids_list = TrainingListPerRound[Round]
        if int(cid) in clients_ids_list:
            net = TabTransformer(num_features).to(DEVICE)
            trainloader = Dataloaders[Round][int(cid)]
            arg_ = Round
            return FlowerClient(cid, net, trainloader, arg_)
        else:
            raise ValueError(f"Client ID {cid} not found in the list for round {Round}")
    return client_fn

<font color='Brown'>***FL Strategy***</font>
---
--- 

In [30]:
Global_Models = {}
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def __init__(self, additional_argument, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.additional_argument = additional_argument
    def aggregate_fit(self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]], failures: List[BaseException]) -> Optional[fl.common.NDArrays]:
        aggregated_parameters_tuple = super().aggregate_fit(rnd, results, failures)
        aggregated_parameters, _ = aggregated_parameters_tuple
        if aggregated_parameters is not None:
            # print(f"Saving round {rnd} aggregated_parameters...")
            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_weights: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)
            # Convert `List[np.ndarray]` to PyTorch`state_dict`
            Global_Models[self.additional_argument] = TabTransformer(num_features)
            params_dict = zip(Global_Models[self.additional_argument].state_dict().keys(), aggregated_weights)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            Global_Models[self.additional_argument].load_state_dict(state_dict, strict=True)
            torch.save(Global_Models[self.additional_argument].state_dict(), f"{PATH}/GlobalModel_{self.additional_argument}.pth")
        return aggregated_parameters_tuple

In [31]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.
    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "current_round": server_round,  # The current round of federated learning
        "local_epochs": EPOCHS #1 if rnd < 2 else 2,  # 
    }
    return config

***Running the Generalized FL Round***
---
---

In [32]:
print("Loading Initial Global Model")
Global_Models[0] = TabTransformer(num_features)
Global_Models[0].load_state_dict(torch.load("0_Input_Random_model_Net.pth"))
Global_Models[0].train()

TrainingListPerRound = {}
for Round in range(1, ROUNDS+1):
    TrainingListPerRound[Round] = []     
    for CLIENT in range (NUM_CLIENTS):
        TrainingListPerRound[Round].append(int(CLIENT))

for Round in range(1, ROUNDS+1):
    print("Starting FL Round: ", Round)
    strategy = SaveModelStrategy(
            fraction_fit=1.0,  # Sample 100% of available clients for training
            fraction_evaluate=0,  # Sample 50% of available clients for evaluation
            min_fit_clients=2,  # Never sample less than 10 clients for training
            min_evaluate_clients=0,  # Never sample less than 5 clients for evaluation
            min_available_clients=2,  # Wait until all 10 clients are available
            on_fit_config_fn=fit_config,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Global_Models[Round-1])),
            additional_argument = Round
    )
    # print(f'Current training nodes at round {Round}: ', len(TrainingListPerRound[Round]), ' ', TrainingListPerRound[Round])
    client_fn = General_Client()
    fl.simulation.start_simulation(
        client_fn=lambda cid: client_fn(cid, int(Round)),
        num_clients=int(NUM_CLIENTS),
        config=fl.server.ServerConfig(num_rounds=int(1)),
        client_resources={"num_cpus":16, "num_gpus":1}, 
        ray_init_args = {'num_cpus': 16, 'num_gpus': 1},
        strategy=strategy
    )
    print("End of FL Round: ", Round)
    print("Loading Global Model: ", Round)
    Global_Models[Round] = TabTransformer(num_features)
    Global_Models[Round].load_state_dict(torch.load(f"{PATH}/GlobalModel_{Round}.pth"))
    Global_Models[Round].train()

INFO flwr 2025-12-02 21:30:40,120 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)


Loading Initial Global Model
Starting FL Round:  1


2025-12-02 21:30:45,128	INFO worker.py:1771 -- Started a local Ray instance.
INFO flwr 2025-12-02 21:30:51,832 | app.py:210 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2714866483.0, 'GPU': 1.0, 'accelerator_type:T4': 1.0, 'node:__internal_head__': 1.0, 'CPU': 16.0, 'memory': 5429732967.0, 'node:10.192.11.33': 1.0}
INFO flwr 2025-12-02 21:30:51,833 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 16, 'num_gpus': 1}
INFO flwr 2025-12-02 21:30:51,854 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO flwr 2025-12-02 21:30:51,855 | server.py:89 | Initializing global parameters
INFO flwr 2025-12-02 21:30:51,856 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2025-12-02 21:30:51,857 | server.py:91 | Evaluating initial parameters
INFO flwr 2025-12-02 21:30:51,857 | server.py:104 | FL starting
DEBUG flwr 2025-12-02 21:30:51,858 | server.py:222 | fit_round 1: strategy sampled 48 client

(DefaultActor pid=107232) [Client 46, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 29, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 34, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 7, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 24, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 5, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 35, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 18, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 39, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=107232) [Client 23, round 1] fit, config: {'current_round': 1, 'local_epochs': 1}
(D

DEBUG flwr 2025-12-02 21:31:50,533 | server.py:236 | fit_round 1 received 48 results and 0 failures
WARNING flwr 2025-12-02 21:31:50,690 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


...

(DefaultActor pid=170946) [Client 36, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 23, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 7, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 32, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 43, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 3, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 11, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 14, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 33, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}
(DefaultActor pid=170946) [Client 2, round 40] fit, config: {'current_round': 1, 'local_epoch

DEBUG flwr 2025-12-02 22:15:28,823 | server.py:236 | fit_round 1 received 48 results and 0 failures
WARNING flwr 2025-12-02 22:15:28,953 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(DefaultActor pid=170946) [Client 1, round 40] fit, config: {'current_round': 1, 'local_epochs': 1}


INFO flwr 2025-12-02 22:15:28,966 | server.py:171 | evaluate_round 1: no clients selected, cancel
INFO flwr 2025-12-02 22:15:28,967 | server.py:153 | FL finished in 54.86793510700045
INFO flwr 2025-12-02 22:15:28,987 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2025-12-02 22:15:28,988 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2025-12-02 22:15:28,988 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2025-12-02 22:15:28,989 | app.py:228 | app_fit: losses_centralized []
INFO flwr 2025-12-02 22:15:28,989 | app.py:229 | app_fit: metrics_centralized {}


End of FL Round:  40
Loading Global Model:  40


<font color='Grey'>***Performance Testing***</font>
---
--- 

In [33]:
import os
import glob

# Define the directory and file pattern
directory = PATH + '/'
pattern = "GlobalModel_*.pth"

# Find all matching files
files = glob.glob(os.path.join(directory, pattern))

# Extract numbers from file names
numbers = []
for file in files:
    base_name = os.path.basename(file)
    num_str = base_name.replace("GlobalModel_", "").replace(".pth", "")
    try:
        numbers.append(int(num_str))
    except ValueError:
        pass

# Determine the maximum number
max_num = max(numbers) if numbers else 0
print(max_num)

# Use the max_num in a loop
for num in range(1, max_num + 1):
    file_path = f"{PATH}/GlobalModel_{num}.pth"
    if os.path.exists(file_path):
        # Load the file or perform any operation you need
        print(f"Loading {file_path}")
    else:
        print(f"File {file_path} does not exist")

40
Loading GlobalOutputs/GlobalModel_1.pth
Loading GlobalOutputs/GlobalModel_2.pth
Loading GlobalOutputs/GlobalModel_3.pth
Loading GlobalOutputs/GlobalModel_4.pth
Loading GlobalOutputs/GlobalModel_5.pth
Loading GlobalOutputs/GlobalModel_6.pth
Loading GlobalOutputs/GlobalModel_7.pth
Loading GlobalOutputs/GlobalModel_8.pth
Loading GlobalOutputs/GlobalModel_9.pth
Loading GlobalOutputs/GlobalModel_10.pth
Loading GlobalOutputs/GlobalModel_11.pth
Loading GlobalOutputs/GlobalModel_12.pth
Loading GlobalOutputs/GlobalModel_13.pth
Loading GlobalOutputs/GlobalModel_14.pth
Loading GlobalOutputs/GlobalModel_15.pth
Loading GlobalOutputs/GlobalModel_16.pth
Loading GlobalOutputs/GlobalModel_17.pth
Loading GlobalOutputs/GlobalModel_18.pth
Loading GlobalOutputs/GlobalModel_19.pth
Loading GlobalOutputs/GlobalModel_20.pth
Loading GlobalOutputs/GlobalModel_21.pth
Loading GlobalOutputs/GlobalModel_22.pth
Loading GlobalOutputs/GlobalModel_23.pth
Loading GlobalOutputs/GlobalModel_24.pth
Loading GlobalOutputs/

In [34]:
pred_test = {}
actual_test = {}
accuracy_test = {}
loss_test = {}
G = 0

for num in range(1, max_num+1):
    model = TabTransformer(num_features)
    model.load_state_dict(torch.load(f"{PATH}/GlobalModel_{num}.pth"))
    model.eval()
    
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    G = G + 1
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in Dataloaders['Test']:
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(Dataloaders['Test'].dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy) 

    pred_test[f'Global_{G}'] = prediction_matrix
    actual_test[f'Global_{G}'] = actual_matrix
    accuracy_test[f'Global_{G}'] = acc_matrix
    loss_test[f'Global_{G}'] = loss_matrix 

    filename = f'{PATH}/Global_{G}_pred'
    outfile = open(filename,'wb')
    pickle.dump(pred_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_actual'
    outfile = open(filename,'wb')
    pickle.dump(actual_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_accurracy'
    outfile = open(filename,'wb')
    pickle.dump(accuracy_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_loss'
    outfile = open(filename,'wb')
    pickle.dump(loss_test[f'Global_{G}'],outfile)
    outfile.close()